In [ ]:
import os
import pickle
import json
import pandas as pd

# Define variables

In [ ]:
CONFIG='/kaggle/input/vinbigdata-mmdet-trainedmodel/config.py'
CHECKPOINT='/kaggle/input/vinbigdata-mmdet-trainedmodel/vfnet.pth'
OUTPUT_DIR='/kaggle/working/vinbig_output'
TEST_ANNOTATIONS_PATH='/kaggle/input/vinbigdatacocotestdataset/test_annotations.json'
TEST_METADATA_PATH='/kaggle/input/vinbigdata-original-image-dataset/vinbigdata/test.csv'

# Install dependencies

In [ ]:
!nvcc -V
!gcc --version

In [ ]:
!pip install mmcv-full

!git clone --branch v2.7.0 https://github.com/open-mmlab/mmdetection.git
    
%cd mmdetection
!pip install -e .

In [ ]:
os.makedirs(OUTPUT_DIR)

# Inference

In [ ]:
!cat $CONFIG

In [ ]:
!python tools/test.py $CONFIG $CHECKPOINT --out results.pkl \
--cfg-options model.test_cfg.score_thr='0.01' model.test_cfg.max_per_img='200'


# Format predictions

In [ ]:
with open('./results.pkl', 'rb') as f:
    data = pickle.load(f)

with open(TEST_ANNOTATIONS_PATH, 'rb') as f:
    ann = json.load(f)

test_df = pd.read_csv(TEST_METADATA_PATH)

file_ids = [file_name.get('file_name').split('.png')[0].split('/')[1] for file_name in ann.get('images')]

In [ ]:
ann_with_pred = zip(file_ids, data)
submission_vals = []

for _id, preds in ann_with_pred:
    boxes = []
    scores = []
    labels = []
    width = test_df[test_df.image_id==_id]['width'].iloc[0]
    height = test_df[test_df.image_id==_id]['height'].iloc[0]  
    for i, pred in enumerate(preds):
        if len(pred):
            for p in pred:
                box = p[:4]
                boxes.append(box)
                score = p[4].astype(float)
                scores.append(score)
                labels.append(i)
    
    _id_preds = []
    for i in range(len(boxes)):
        _id_preds.append(str(labels[i]))
        _id_preds.append(str(scores[i]))
        _id_preds.append(str(boxes[i][0]))
        _id_preds.append(str(boxes[i][1]))
        _id_preds.append(str(boxes[i][2]))
        _id_preds.append(str(boxes[i][3]))
    pred_str = " ".join(_id_preds)
    
    submission_vals.append([_id, pred_str])

# Create submission file

In [ ]:
df = pd.DataFrame(submission_vals, columns = ['image_id','PredictionString'])
df.to_csv("submission.csv",index=False)
df.head()